# Proyecto Final del curso de Ingeniería de Datos 

Se propone crear un pipeline que extraiga datos de una API pública de forma constante combinándolos con información extraída de una base de datos y colocándolos en un Data Warehouse.

### Conexion con la API

In [42]:
# Libreria para interactuar con APIs
import requests

import pandas as pd

In [4]:
# Datita de como funciona el metodo get
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      c:\users\hp\appdata\local\programs\python\python310\lib\site-packages\requests\api.py
Type:      function

In [56]:
base_url = "https://apitransporte.buenosaires.gob.ar"
endpoint = "colectivos/vehiclePositions?client_id=33987e066c34484585f7d1c725e12831&client_secret=B72247DC28B7432BBCa552ED24E894C5&json=1"

full_url = f"{base_url}/{endpoint}"

r = requests.get(full_url)

In [57]:
r.status_code

200

In [58]:
json_data = r.json()
json_data

{'_entity': [{'_alert': None,
   '_id': '1',
   '_is_deleted': False,
   '_trip_update': None,
   '_vehicle': {'_congestion_level': 0,
    '_current_status': 2,
    '_current_stop_sequence': 15,
    '_occupancy_status': 0,
    '_position': {'_bearing': 0,
     '_latitude': -34.64196,
     '_longitude': -58.5727,
     '_odometer': 32274,
     '_speed': 9.444444,
     'extensionObject': None},
    '_stop_id': '6568117976',
    '_timestamp': 1706294690,
    '_trip': {'_direction_id': 0,
     '_route_id': '1468',
     '_schedule_relationship': 0,
     '_start_date': '20240126',
     '_start_time': '15:21:00',
     '_trip_id': '95431-1',
     'extensionObject': None},
    '_vehicle': {'_id': '1827',
     '_label': '3143-923',
     '_license_plate': None,
     'extensionObject': None},
    'extensionObject': None},
   'extensionObject': None},
  {'_alert': None,
   '_id': '2',
   '_is_deleted': False,
   '_trip_update': None,
   '_vehicle': {'_congestion_level': 0,
    '_current_status': 2,


In [59]:
type(json_data)

dict

In [60]:
json_data.keys()

dict_keys(['_entity', '_header', 'extensionObject'])

Para pasar a un dataframe la data


In [66]:
# Para pasar el json a una dataframe
df = pd.json_normalize(json_data['_entity'])
df.head()

,_alert,_id,_is_deleted,_trip_update,extensionObject,_vehicle._congestion_level,_vehicle._current_status,_vehicle._current_stop_sequence,_vehicle._occupancy_status,_vehicle._position._bearing,...,_vehicle._trip._start_date,_vehicle._trip._start_time,_vehicle._trip._trip_id,_vehicle._trip.extensionObject,_vehicle._vehicle._id,_vehicle._vehicle._label,_vehicle._vehicle._license_plate,_vehicle._vehicle.extensionObject,_vehicle.extensionObject,_vehicle._trip
0,None,1,False,None,None,0,2,15,0,0,...,20240126,15:21:00,95431-1,NaN,1827,3143-923,None,None,None,NaN
1,None,2,False,None,None,0,2,50,0,0,...,20240126,14:45:00,95428-1,NaN,1833,3137-923,None,None,None,NaN
2,None,3,False,None,None,0,2,10,0,0,...,20240126,15:30:00,95357-1,NaN,1838,3132-922,None,None,None,NaN
3,None,4,False,None,None,0,2,36,0,0,...,20240126,14:45:00,95428-1,NaN,1845,3138-923,None,None,None,NaN
4,None,5,False,None,None,0,2,17,0,0,...,20240126,15:30:00,95357-1,NaN,1849,3126-922,None,None,None,NaN
